# 微調 Open AI 模型

此筆記本基於 Open AI 目前在 [微調](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) 文件中提供的指引。

微調透過使用與特定用例或場景相關的額外數據和上下文重新訓練基礎模型，以提升其在您的應用中的表現。請注意，像是 _few shot learning_ 和 _retrieval augmented generation_ 這類提示工程技術，允許您使用相關數據來增強預設提示以提升品質。然而，這些方法受限於目標基礎模型的最大 token 視窗大小。

透過微調，我們實際上是使用所需數據重新訓練模型本身（允許我們使用比最大 token 視窗能容納更多的範例）— 並部署一個 _自訂_ 版本的模型，該版本在推理時不再需要提供範例。這不僅提升了我們提示設計的效能（我們在使用 token 視窗時有更多彈性），也可能降低成本（減少推理時需要傳送給模型的 token 數量）。

微調有四個步驟：
1. 準備訓練數據並上傳。
1. 執行訓練工作以獲得微調模型。
1. 評估微調模型並反覆調整以提升品質。
1. 在滿意後部署微調模型進行推理。

請注意，並非所有基礎模型都支援微調—[請參閱 OpenAI 文件](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) 以獲取最新資訊。您也可以微調先前已微調過的模型。在本教學中，我們將使用 `gpt-35-turbo` 作為我們微調的目標基礎模型。

---


### 第 1.1 步：準備您的數據集

讓我們建立一個聊天機械人，通過用打油詩回答有關元素的問題，幫助您了解元素週期表。在_這個_簡單的教程中，我們將僅創建一個數據集，用幾個示例回應來訓練模型，這些示例展示了數據的預期格式。在實際應用中，您需要創建一個包含更多示例的數據集。如果存在開放數據集（針對您的應用領域），您也可以使用並重新格式化它以用於微調。

由於我們專注於`gpt-35-turbo`並尋求單輪回應（聊天完成），我們可以使用[此建議格式](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)來創建示例，該格式符合 OpenAI 聊天完成的要求。如果您預期多輪對話內容，則應使用[多輪示例格式](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)，其中包含一個`weight`參數，用以標示哪些訊息應該在微調過程中使用（或不使用）。

在本教程中，我們將使用較簡單的單輪格式。數據採用[jsonl 格式](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst)，每行一條記錄，每條記錄表示為 JSON 格式的物件。以下片段展示了兩條記錄作為示例——完整示例集（10 個示例）請參見[training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl)，我們將用於微調教程。**注意：**每條記錄_必須_定義在單行中（不能像格式化的 JSON 文件那樣跨行拆分）

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

在實際應用中，您將需要更多的示例集以獲得良好效果——權衡點在於回應質量與微調所需的時間/成本。我們使用較小的示例集，以便快速完成微調，從而說明整個過程。請參閱[這個 OpenAI Cookbook 範例](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)以獲取更複雜的微調教程。


---

### 第 1.2 步 上傳您的數據集

使用 Files API [如此處所述](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) 上傳數據。請注意，為了執行此代碼，您必須先完成以下步驟：
 - 安裝 `openai` Python 套件（確保您使用的版本 >=0.28.0 以獲取最新功能）
 - 將 `OPENAI_API_KEY` 環境變量設置為您的 OpenAI API 金鑰
欲了解更多，請參閱課程提供的[設定指南](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)。

現在，執行代碼以從您的本地 JSONL 文件創建一個上傳文件。


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Step 2.1: 使用 SDK 建立微調工作


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### 第 2.2 步：檢查工作的狀態

以下是您可以使用 `client.fine_tuning.jobs` API 做的一些事情：
- `client.fine_tuning.jobs.list(limit=<n>)` - 列出最近的 n 個微調工作
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 獲取特定微調工作的詳細資訊
- `client.fine_tuning.jobs.cancel(<job_id>)` - 取消微調工作
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - 列出該工作最多 n 個事件
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

流程的第一步是_驗證訓練文件_，以確保資料格式正確。


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### 第 2.3 步：追蹤事件以監察進度


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 第2.4步：在OpenAI儀表板查看狀態


你亦可以透過瀏覽 OpenAI 網站並探索平台的 _Fine-tuning_ 部分來查看狀態。這會顯示你當前工作的狀態，並讓你追蹤之前工作執行的歷史。在這張截圖中，你可以看到之前的執行失敗，而第二次執行成功。作為背景說明，這是因為第一次執行時使用了格式錯誤的 JSON 檔案記錄 — 修正後，第二次執行成功完成，並使模型可供使用。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.mo.png)


你亦可以透過在視覺化儀表板中向下滾動，查看狀態訊息及指標，如下所示：

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.mo.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.mo.png)|


---

### 第 3.1 步：在程式碼中取得 ID 並測試微調模型


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### 第 3.2 步：在 Playground 載入及測試微調模型

你現在可以用兩種方法測試微調模型。首先，你可以訪問 Playground，並使用 Models 下拉選單從列出的選項中選擇你新微調的模型。另一個選項是使用微調面板中顯示的「Playground」選項（見上方截圖），它會啟動以下的 _比較_ 視圖，並將基礎模型與微調模型版本並排顯示，方便快速評估。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.mo.png)

只需填寫你訓練數據中使用的系統上下文，並提供你的測試問題。你會注意到兩邊都會更新相同的上下文和問題。執行比較後，你將看到它們輸出結果的差異。_注意微調模型如何以你範例中提供的格式呈現回應，而基礎模型則僅遵循系統提示_。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.mo.png)

你會注意到比較還會提供每個模型的 token 數量，以及推理所花費的時間。**這個特定範例是簡化示範流程用，並不反映真實世界的數據集或情境**。你可能會發現兩個範例顯示相同的 token 數（系統上下文和使用者提示相同），但微調模型推理時間較長（自訂模型）。

在真實世界情境中，你不會使用像這樣的玩具範例，而是會針對真實數據（例如客戶服務的產品目錄）進行微調，回應品質會更明顯。在 _那個_ 情境下，要用基礎模型達到相同的回應品質，將需要更多自訂提示工程，這會增加 token 使用量，並可能增加推理的相關處理時間。_想試試看，可以參考 OpenAI Cookbook 中的微調範例開始。_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：
本文件係使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我哋致力於確保準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件之母語版本應視為權威來源。對於重要資訊，建議採用專業人工翻譯。我哋對因使用本翻譯而引起之任何誤解或誤釋概不負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
